In [7]:
import os
from datetime import datetime
import torch
from models.resnet18 import ResNet

In [8]:
num_labels = 8
#train = False
device = "cpu"
onnx_name = "FacialExpression"
dropout = 0.1
fm_distillation = True

class ResNetConfig:
	def __init__(self):
		self.fm_distillation = fm_distillation
		self.dropout = dropout
		self.num_labels = num_labels


In [9]:
# ResNet18
model_config = ResNetConfig()
model = ResNet(model_config)
ckpt_name = os.path.join("checkpoints_fm_resnet", "AffectNet", "resnet.pt")

In [10]:
checkpoints = torch.load(ckpt_name, map_location=torch.device(device))["model"]
model.load_state_dict(checkpoints, strict=True)
torch.no_grad()
model.eval()

ResNet(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)


In [11]:

"""
for images, labels in loader:
	images = images.to(device)
	labels = labels.to(device)
	labels_pred = model(images)
	labels_pred = torch.clamp(labels_pred, min=0.0, max=5.0)
"""

dummy_input = torch.rand((1, 3, 224, 224), device=device)
input_names = [ "image" ]
output_names = [ "FEs" ]
onnx_name = onnx_name + datetime.now().strftime("%Y%m%d%H%M%S") + ".onnx"

torch.onnx.export(
	model, 
	dummy_input, 
	onnx_name, 
	verbose=True, 
	input_names=input_names,
	output_names=output_names
)

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [12]:
import onnx
model = onnx.load(onnx_name)
onnx.checker.check_model(model)
print(onnx.helper.printable_graph(model.graph))

graph torch_jit (
  %image[FLOAT, 1x3x224x224]
) initializers (
  %classifier.0.weight[FLOAT, 128x512]
  %classifier.0.bias[FLOAT, 128]
  %classifier.2.weight[FLOAT, 128]
  %classifier.2.bias[FLOAT, 128]
  %classifier.2.running_mean[FLOAT, 128]
  %classifier.2.running_var[FLOAT, 128]
  %classifier.4.weight[FLOAT, 8x128]
  %classifier.4.bias[FLOAT, 8]
  %onnx::Conv_211[FLOAT, 64x3x7x7]
  %onnx::Conv_212[FLOAT, 64]
  %onnx::Conv_214[FLOAT, 64x64x3x3]
  %onnx::Conv_215[FLOAT, 64]
  %onnx::Conv_217[FLOAT, 64x64x3x3]
  %onnx::Conv_218[FLOAT, 64]
  %onnx::Conv_220[FLOAT, 64x64x3x3]
  %onnx::Conv_221[FLOAT, 64]
  %onnx::Conv_223[FLOAT, 64x64x3x3]
  %onnx::Conv_224[FLOAT, 64]
  %onnx::Conv_226[FLOAT, 128x64x3x3]
  %onnx::Conv_227[FLOAT, 128]
  %onnx::Conv_229[FLOAT, 128x128x3x3]
  %onnx::Conv_230[FLOAT, 128]
  %onnx::Conv_232[FLOAT, 128x64x1x1]
  %onnx::Conv_233[FLOAT, 128]
  %onnx::Conv_235[FLOAT, 128x128x3x3]
  %onnx::Conv_236[FLOAT, 128]
  %onnx::Conv_238[FLOAT, 128x128x3x3]
  %onnx::Conv_2